In [2]:
import yfinance as yf
import numpy as np
import pandas as pd
import pywt
import matplotlib.pyplot as plt
import os
from datetime import datetime

In [ ]:
# Function to download stock data
def download_stock_data(stocks, start_date, end_date):
    data = yf.download(stocks, start=start_date, end=end_date, group_by='ticker', auto_adjust=True)
    return {ticker: data[ticker]['Close'].dropna() for ticker in stocks}

# Function to create segments
def create_segments(data, window_size=100, overlap=0.5):
    step = int(window_size * (1 - overlap))  # Calculate step size based on overlap
    segments = []
    for ticker, series in data.items():
        series = series.to_numpy()
        for i in range(0, len(series) - window_size + 1, step):
            segment = series[i:i + window_size]
            segments.append({'ticker': ticker, 'segment': segment})
    return segments

# Function to generate and save spectrograms
def generate_and_save_spectrograms(segments, output_dir, wavelet='cmor', scale_range=(1, 64)):
    for idx, segment_data in enumerate(segments):
        ticker = segment_data['ticker']
        segment = segment_data['segment']
        
        # Create folder for each stock if it doesn't exist
        ticker_dir = os.path.join(output_dir, ticker)
        os.makedirs(ticker_dir, exist_ok=True)
        
        # Generate Continuous Wavelet Transform (CWT)
        scales = np.arange(scale_range[0], scale_range[1])
        coefficients, frequencies = pywt.cwt(segment, scales, wavelet)
        
        # Save the spectrogram
        plt.figure(figsize=(8, 6))
        plt.imshow(
            np.abs(coefficients), 
            extent=[0, len(segment), scale_range[0], scale_range[1]], 
            cmap='jet', 
            aspect='auto', 
            origin='lower'
        )
        plt.colorbar(label='Magnitude')
        plt.title(f"Spectrogram - {ticker} - Segment {idx}")
        plt.xlabel("Time")
        plt.ylabel("Scale")
        
        # Save the image
        filename = os.path.join(ticker_dir, f"{ticker}_segment_{idx}.png")
        plt.savefig(filename)
        plt.close()

# Define stock names and date ranges
stocks = ["MSTR", "AAPL", "TSLA", "INTC", "MCD", "KO", "NKE"]
train_start_date = "2003-01-01"
train_end_date = "2016-12-31"
test_start_date = "2017-01-01"
test_end_date = "2019-12-31"

# Directories for saving spectrograms
train_output_dir = "/Users/roberto/Desktop/SPECTOGRAMSSS/train"
test_output_dir = "/Users/roberto/Desktop/SPECTOGRAMSSS/test"

# Download data for training and testing
train_data = download_stock_data(stocks, train_start_date, train_end_date)
test_data = download_stock_data(stocks, test_start_date, test_end_date)

# Generate segments for training and testing
train_segments = create_segments(train_data, window_size=100, overlap=0.5)
test_segments = create_segments(test_data, window_size=100, overlap=0.5)

# Generate and save spectrograms
generate_and_save_spectrograms(train_segments, train_output_dir)
generate_and_save_spectrograms(test_segments, test_output_dir)

[*********************100%***********************]  7 of 7 completed
[*********************100%***********************]  7 of 7 completed
/Users/roberto/.pyenv/versions/3.11.6/lib/python3.11/site-packages/pywt/_cwt.py:117: FutureWarning: Wavelets from the family cmor, without parameters specified in the name are deprecated. The name should follow the format cmorB-C, where B and C are floats representing the bandwidth frequency and center frequency, respectively (example, for backward compatibility: cmor = cmor1.0-0.5).
  wavelet = DiscreteContinuousWavelet(wavelet)
